# Demo of policy guard component
This Jupyter notebook is a demo of the Policy Guard ALTK Component.

This approach is effective when agent responses are expected to conform to some set of semantic or syntactic guidelines. 


In [ ]:
from altk.toolkit_core.llm import get_llm
from altk.toolkit_core.core.toolkit import ComponentConfig

from altk.policy_guard_toolkit.core.toolkit import (
    PolicyDetectorInput,
    PolicyRepairerInput,
    RepairConfig,
)

from altk.policy_guard_toolkit.detect.detector import BatchPolicyLLMDetector
from altk.policy_guard_toolkit.repair.repairer import BatchPolicyRepairer

The inputs to this component are: a user query (`user_query`), a response to the query (from a model, agent, or human), and a set of policies that the response is expected to follow. 

In [ ]:
user_query = "Can you provide a summary of the major announcements made during the 2022 Google I/O conference?"

policy_list = [
    "In your entire response, refrain from the use of any commas.",
    "Answer with at least 200 words.",
    "Your answer must contain a title, wrapped in double angular brackets, such as <<poem of joy>>.",
    "In your response, the word Android should appear less than 5 times.",
    "Your answer must contain exactly 6 bullet points. Use the markdown bullet points such as:\n* This is point 1. \n* This is point 2",
    "Include one of the following options in your answer: ('My answer is yes.', 'My answer is no.', 'My answer is maybe.')",
    "In your response, words with all capital letters should appear at least 7 times.",
    "Begin your response with this exact phrase: `During the 2022`. No other words should come before this phrase.",
    "Paragraph 2 must have the word 'opportunity' in its first three words. Paragraphs and only paragraphs are separated with each other by two new lines as if it was '\\n\\n' in python. ",
    "Include keywords ['Android', 'Bard'] in the response.",
]

response = "During the 2022 Google I/O conference was a major event that took place in MAY and JUNE of that year and it was a great SUCCESS with many EXCITING announcements made by the company's TOP officials including SUNDAR PICHAI and other HIGH-PROFILE individuals from the TECH industry. The conference was a huge OPPORTUNITY for Google to showcase its LATEST innovations and PRODUCTS including the NEW Android operating system and other SOFTWARE updates. The event was attended by THOUSANDS of developers and TECH enthusiasts from around the WORLD who were eager to learn about the LATEST trends and ADVANCES in the field of TECHNOLOGY. The conference featured many KEYNOTE speeches and PRESENTATIONS by EXPERTS in the field of ARTIFICIAL INTELLIGENCE and MACHINE LEARNING. My answer is yes.\n\n\nOpportunity opportunity opportunity to learn about the LATEST developments in the field of TECHNOLOGY was one of the main reasons why many people attended the conference. The event featured many EXHIBITS and DEMONSTRATIONS of NEW PRODUCTS and SERVICES including the Bard which is a CONVERSATIONAL AI model developed by Google. The conference also provided a great opportunity for developers to NETWORK and LEARN from each other and to get feedback on their WORK from EXPERTS in the field. The event was a huge SUCCESS and it helped to establish Google as a LEADER in the TECH industry. The company's COMMITMENT to INNOVATION and EXCELLENCE was evident in the many ANNOUNCEMENTS and PRESENTATIONS made during the conference. The NEW Android operating system was one of the main topics of discussion during the event.\n\n\n<<Google I/O Conference Summary>>\nThe conference featured many HIGH-PROFILE speakers and PRESENTATIONS including KEYNOTE speeches by SUNDAR PICHAI and other TOP officials from Google. The event was a great SUCCESS and it helped to establish the company as a LEADER in the TECH industry. Here are some of the major announcements made during the conference:\n* The NEW Android operating system was unveiled during the event and it features many NEW and EXCITING features including IMPROVED PERFORMANCE and SECURITY.\n* The Bard which is a CONVERSATIONAL AI model developed by Google was also showcased during the conference and it has the potential to REVOLUTIONIZE the way we interact with TECHNOLOGY.\n* The conference featured many EXHIBITS and DEMONSTRATIONS of NEW PRODUCTS and SERVICES including the LATEST advancements in ARTIFICIAL INTELLIGENCE and MACHINE LEARNING.\n* The event provided a great opportunity for developers to NETWORK and LEARN from each other and to get feedback on their WORK from EXPERTS in the field.\n* The NEW Android operating system is expected to be released later this year and it will be available on many DIFFERENT devices including SMARTPHONES and TABLETS.\n* The conference was a huge SUCCESS and it helped to establish Google as a LEADER in the TECH industry with a strong COMMITMENT to INNOVATION and EXCELLENCE and a focus on DEVELOPING NEW and EXCITING PRODUCTS and SERVICES including the Bard and the NEW Android operating system."

Configure the LLM you will use in the detector and repairer components. 

In [ ]:
import os

os.environ["WX_API_KEY"] = "your_wx_api_key"
os.environ["WX_PROJECT_ID"] = "your_wx_project_id"

WATSONX_CLIENT = get_llm("watsonx")
llm_client = WATSONX_CLIENT(
    model_id="meta-llama/llama-3-3-70b-instruct",
    api_key=os.getenv("WX_API_KEY"),
    project_id=os.getenv("WX_PROJECT_ID"),
    url=os.getenv("WX_URL", "https://us-south.ml.cloud.ibm.com"),
)
config = ComponentConfig(llm_client=llm_client)

Initialize the detector and repairer component. 

In [ ]:
detector = BatchPolicyLLMDetector(name="batch_policy_llm_detector", config=config)
repairer = BatchPolicyRepairer(
    name="batch_policy_llm_repairer", detector=detector, config=config
)

The user query and agent response is passed to the detector component which identifies whether there is a violation of one of the given policies. This allows the agent builder to implement a go/no-go decision on the response, to raise a warning, or to attempt a repair. 

In [ ]:
# Set up the input dataclass.
input_example = PolicyDetectorInput(
    policies=policy_list,
    prompt=user_query,
    response=response,
)

# Make detection call
detection = detector.detect(input_example)
# Boolean flags indicating whether the response complied with each policy in the list.
detected_compliance = [d.compliance for d in detection.policy_outputs]

print("\n" + "++++++ DETECTED COMPLIANCE: ", detected_compliance)

If there is a detected violation, the repairer can use it to produce an alternative response that is compliant with the policies. 

In [ ]:
repair_inputs = PolicyRepairerInput(
    config=RepairConfig(),  # Use default repair params
    detection_input=input_example,
    detection_output=detection,
)

repair = repairer.repair(repair_inputs)
repair_response = repair.repaired_text

print("\n++++++ REPAIRED RESPONSE:\n", repair_response)